Problem Statement
Classification of Web page content is vital to many tasks in Web information retrieval such as maintaining Web directories and focused crawling. The uncontrolled nature of Web content presents additional challenges to Web page classification as compared to traditional text classification, however the interconnected nature of hypertext also provides features that can assist the process.

Here the task is to classify the web pages to the respective classes it belongs to, in a single label classification setup (Each webpage can belong to only 1 class).

Basically given the complete html and url, predict the tag a web page belongs to out of 9 predefined tags as given below:

  1) People profile
  2) Conferences/Congress
  3) Forums
  4) News article
  5) Clinical trials
  6) Publication
  7) Thesis
  8) Guidelines
  9) Others
 

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#unzip Autopilot folder as data is inside there
!cp  /content/drive/Hackthon/Innoplexus/* .
#!cp new_html.csv  /content/drive/Hackthon/Innoplexus/

In [6]:
!unzip train.zip

Archive:  train.zip
  inflating: train.csv               
  inflating: html_data.csv           


In [20]:
!ls

datalab			   Innoplexus_train_02.ipynb	  train.csv
drive			   new_html.csv			  train.zip
html_data.csv		   sample_submission_poy1UIu.csv
Innoplexus_train_01.ipynb  test_nvPHrOx.csv


In [2]:
import numpy as np
import pandas as pd
import os, shutil
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Input,ELU
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import pi
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense, Dropout,LeakyReLU,BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from keras.models import load_model
from keras import regularizers
from keras.optimizers import Adam, Adamax, Adagrad
from numpy import argmax
import re

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
             Data = pd.read_csv("train.csv")
Data_Test = pd.read_csv("test_nvPHrOx.csv")

In [4]:
Data.head()

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news


In [5]:
Data.count()
# there are no missing values

Webpage_id    53447
Domain        53447
Url           53447
Tag           53447
dtype: int64

In [11]:
Data['Tag'].value_counts()
#its lokks mode biased towords other, and less data in can see decreasing ordered

others            17417
news               7992
publication        7705
profile            5196
conferences        4666
forum              4503
clinicalTrials     2839
thesis             1800
guidelines         1329
Name: Tag, dtype: int64

In [22]:
s = Data.Url.iloc[1]
s

'http://www.fiercepharma.com/pharma/novo-equipped-to-weather-storm-u-s-diabetes-market-ceo-says'

In [19]:
print(Data_Test.Webpage_id.iloc[-1])
print(Data.Webpage_id.iloc[-1])
print(Data_Test.shape)
print(Data.shape)




79273
79345
(25787, 3)
(53447, 4)


In [38]:
ss =re.findall(r'http://www.\w+.com/(\w.*)',s)
ss

['pharma/novo-equipped-to-weather-storm-u-s-diabetes-market-ceo-says']

In [41]:
#re.findall(r"\w\w+",ss[0])

re.findall(r"\w\w+","dsfs 200 fdsf2000 -f")

['dsfs', '200', 'fdsf2000']

In [0]:
import pandas as pd
import re

In [0]:
f = open("html_data.csv",'r',encoding='utf8')
list1=[]
#7246934572
for i in range(73):
  s = f.read(100000000)
  tt =re.findall(r'\n(\d+),',s,re.M)
  list1.extend(tt)
f.close()

In [12]:
len(list1)

79806

In [15]:
set1=set(list1)
list1=list(set1)
list1.sort()
len(list1)


79354

In [41]:
import pandas as pd
import numpy as np
import re

f = open("html_data.csv",'r',encoding='utf8')
#f = open("temp.csv", 'r', encoding='utf8')
d = {}
count =2
check=False
for i in range(1):
    print(i)
    print(len(d))
    if count==79345:
        break
    s = f.read(100000000)
    s = s.lower()
    for j in range(79345):
        #print(count)
        m1 = s.find('"\n{0},'.format(str(count)))
        #print(m1)
        if m1==-1:
            if check:
                d[count]=''
                count+=1
                check=False
                continue
            check = True
            break
        tt1 = s.find("<title>", m1)
        tt2 = s.find("</title>", tt1)
        if tt1==-1 or tt2 == -1:
            d[count]=''
            count+=1
            break
        sub = s[tt1+7:tt2]
        sub = sub.strip()
        #sub = re.sub("\W",'',sub)
        d[count]=sub
        count+=1
        check=False

print(len(d))
#print(d)
f.close()
print("finished")


0
0
1551
finished


In [0]:
d[1]="Tecfidera, Gilenya and Aubagio&#039;s 3-way battle for MS share is about to heat up | FiercePharma"

In [23]:
len(d)

4384

In [42]:
d

{2: 'novo equipped to weather the storm in the u.s. diabetes market, ceo says | fiercepharma',
 3: 'another exec departs troubled endo--and this time, it&#039;s for another drugmaker | fiercepharma',
 4: 'would teva buy korea&#039;s celltrion to beef up in biosimilars? it wouldn&#039;t say no | fiercepharma',
 5: 'restasis-maker allergan recruits actress marisa tomei to drive dry eye awareness | fiercepharma',
 6: 'nice backs &#039;less-effective&#039; otezla to give psoriatic arthritis patients a pill option | fiercepharma',
 7: 'mylan takes on punching-bag role again as clinton aims to shame &#039;bad actors&#039; | fiercepharma',
 8: 'failed eylea combo trial throws a wrench in regeneron&#039;s expansion effort | fiercepharma',
 9: 'should gsk investors be disgruntled about its insider ceo pick? if they want big changes, yes | fiercepharma',
 10: 'a silver lining to the drug-pricing uproar? for generic competition, yes, gpha chief says | fiercepharma',
 11: 'fda skepticism greets pf

In [34]:
c = str('中国临床试验注册中心世界卫生组织国际临床试验注册平台一级注册机构')
re.sub("!\ww+",'',c)

'中国临床试验注册中心世界卫生组织国际临床试验注册平台一级注册机构'

In [43]:
key = list(d.keys())
values = list(d.values())
type(key)
type(values)

list

In [0]:
dict_new = {"Webpage_id":key,"Title":values}
new_data = pd.DataFrame(dict_new)

In [55]:
#new_data.Title.iloc[78].replace(r'[^\x00-\x7F]+', '')
new_data.Title.apply(lambda row: row.encode('ascii',errors='ignore').decode())

0       novo equipped to weather the storm in the u.s....
1       another exec departs troubled endo--and this t...
2       would teva buy korea&#039;s celltrion to beef ...
3       restasis-maker allergan recruits actress maris...
4       nice backs &#039;less-effective&#039; otezla t...
5       mylan takes on punching-bag role again as clin...
6       failed eylea combo trial throws a wrench in re...
7       should gsk investors be disgruntled about its ...
8       a silver lining to the drug-pricing uproar? fo...
9       fda skepticism greets pfizer&#039;s latest bid...
10      allergan&#039;s vraylar needs &#039;negative&#...
11      how to fight pharma&#039;s pricing scandals? t...
12      did someone say free epipens? mylans distribut...
13      valeant brings in zoetis, warner chilcott vet ...
14      valeant tweaks exec ranks again as it looks fo...
15      hefty orkambi commercial spend helps vertex pi...
16      &#039;sunshine&#039; hits europe: pharma&#039;...
17      frustr

In [56]:
t[70: 90]

70                             clinical trials register
71                                                   - 
72                                                   - 
73                             clinical trials register
74                             clinical trials register
75    isrctn - isrctn16006202: intensified household...
76                             clinical trials register
77                             clinical trials register
78                                                   - 
79                             clinical trials register
80                             clinical trials register
81                             clinical trials register
82                             clinical trials register
83                             clinical trials register
84                             clinical trials register
85                             clinical trials register
86                        umin clinical trials registry
87                        umin clinical trials r

In [6]:
new_data.shape

(1, 2)

In [0]:
new_data.to_csv('new_data.csv',index=False)

#!cp new_data.csv /content/drive/Hackthon/Innoplexus/

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G  6.3G  334G   2% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G  249M  6.2G   4% /opt/bin
/dev/sda1       365G  8.1G  357G   3% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [0]:
!cp new_data.csv /content/drive/Hackthon/Innoplexus/

In [0]:
f = open("html_data.csv",'r',encoding='utf8')
list2=[]
#7246934572
for i in range(73):
  s = f.read(100000000)
  tt =re.findall(r'<title>(\w.*)</title>',s)
  list2.extend(tt)
f.close()


In [7]:
f = open("html_data.csv",'r',encoding='utf8')
list1=[]
#7246934572
for i in range(73):
  s = f.read(100000000)
  tt =re.findall(r'"\n\d+.*',s,re.M)
  list1.extend(tt)
f.close()


NameError: ignored

In [55]:
len(list2)

77898

In [0]:
f = open("html_data.csv",'r',encoding='utf8')
s = f.read(1000000)
f.close()

In [0]:
list3 = re.findall(r"<html.*/html>",s,re.DOTALL)

In [77]:
list3[:30]

['<html lang=""en"" dir=""ltr"" xmlns:article=""http://ogp.me/ns/article#"" xmlns:book=""http://ogp.me/ns/book#"" xmlns:product=""http://ogp.me/ns/product#"" xmlns:profile=""http://ogp.me/ns/profile#"" xmlns:video=""http://ogp.me/ns/video#"" prefix=""content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# "">\n  <head>\n    <meta charset=""utf-8"" /><script type=""text/javascript"">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if(""function""==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:

In [73]:
tt =re.findall(r'\n(\d+),',s,re.M)
len(tt)

16